<a href="https://colab.research.google.com/github/dartwinshu/dibimbing-introduction-to-machine-learning/blob/main/Dibimbing%20-%20Introduction%20to%20Machine%20Learning%20-%20Part%2002%20(Clustering).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Load Package

In [ ]:
# install yellowbrick untuk visualisasi klaster dan mencari klaster optimal
!pip install yellowbrick

In [ ]:
import numpy as np # untuk manipulasi objek array dan fungsi aritmatika lainnya
import pandas as pd # untuk menipulasi objek dataframe
import matplotlib.pyplot as plt # untuk visualisasi data
import seaborn as sns # untuk visualisasi data yang jauh lebih mudah

# untuk melakukan standarisasi pada data numerik
from sklearn.preprocessing import StandardScaler
# untuk pemodelan k-means
from sklearn.cluster import KMeans
# untuk mereduksi dimensi
from sklearn.decomposition import PCA
# untuk visualisasi elbow method
from yellowbrick.cluster import KElbowVisualizer
# untuk visualisasi silhouette score
from yellowbrick.cluster import SilhouetteVisualizer
# untuk mencari klaster optimal
from yellowbrick.cluster import KElbowVisualizer

# Load Dataset

In [ ]:
raw_data = pd.read_csv("https://bit.ly/customer_segmentation_bank_dataset")

In [ ]:
# cek dataset
raw_data.head()

## Informasi terkait dataset

Dataset berikut adalah sekumpulan data yang dikumpulkan dari berbagai wilayah di USoleh sebuah bank yang terdiri dari profil nasabah dan beberapa metrik terkait kegiatan nasabah. Bank ingin memasarkan sebuah layanan terbaru yang diharapkan dapat digunakan oleh nasabah lama. Namun sebelum memasarkan mereka ingin mengetahui profil customer yang akan menjadi target pemasaran. Beberapa variabel yang digunkan dalam analisis diantaranya:

- Customer ID
- Age
- Edu
- Years
- Income
- Card Debt
- Other Debt
- Defaulted
- Address
- Debt Income Ratio

# Data Inspection & Preprocessing

In [ ]:
# cek struktur data
raw_data.info()

In [ ]:
# cek apakah ada dataset yang hilang
raw_data.isna().sum()

In [ ]:
# mengecek data yang hilang
raw_data[raw_data['Defaulted'].isna()]

Ketika ada nilai yang hilang pada dataset maka sebelum ada perlakuan tertentu untuk mengatasi masalah nilai yang hilang tersebut, diantaranya perlu diketahui apakah:

1. Data hilang secara acak, yaitu kasus dimana data yang hilang diketahui sebabnya dan hilangnya data terjadi secara acak. Sebagai contoh beberapa baris data yang memiliki nilai null pada sensor dikarenakan listrik tidak stabil sehingga data tidak terbaca secara teratur.
2. Data hilang secara tidak acak, yaitu kasus dimana nilai yang hilang pada data diketahui sebabnya dan tidak terjadi secara acak, biasanya karena disebabkan oleh variabel atau kejadian tertentu. Sebagai contoh, terdapat data bernilai null pada variabel total pembayaran karena pelanggan adalah pelanggan baru yang belum membayar tagihan apapun.
3. Data hilang secara acak sebenar-benarnya, yaitu kasus dimana data yang hilang tidak diketahui sama sekali sebab hilangnya dan hal ini terjadi secara acak. 

Pada kasus di atas kita mengasumsikan bahwa data hilang secara acak sebenar-benarnya. Pada kasus ini kita bisa memilih antara membuang data tersebut atau melakukan imputasi pada data. Namun, kita akan memilih untuk membuang data untuk menyederhanakan pengerjaan.

In [ ]:
# membuang nilai null dari raw_data
raw_data = raw_data[raw_data["Defaulted"].isna() == False]

In [ ]:
# cek kembali nilai null pada raw_data
raw_data.isna().sum()

Selanjutnya kita akan melakukan preprocessing pada data

In [ ]:
# membuang variabel 'Address'
raw_data = raw_data.drop('Address', axis = 1)

# menjadikan customer id sebagai index
raw_data = raw_data.set_index('Customer Id')

# cek kembali raw_data
raw_data.head()

Selanjutnya melakukan scaling pada semua data bernilai numerik.

In [ ]:
# lakukan scaling pada data
data_for_model = StandardScaler().fit_transform(raw_data)

In [ ]:
# cek data_for_model
data_for_model

# Modeling

Membuat model K-means

In [ ]:
# menentukan jumlah kelompok data yang akan dibentuk
num_clusters = 3

# fit k-means model pada data
k_means = KMeans(n_clusters=num_clusters, random_state = 1000)
k_means.fit(data_for_model)

# label cluster
labels = k_means.labels_

# cek labels
print(labels)

# Post-Modeling Analysis

In [ ]:
# membuat kolom 'cluster' pada raw_data
raw_data['cluster'] = labels

# cek raw_data
raw_data.head()

In [ ]:
# mengecek distribusi age dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Age')

In [ ]:
# mengecek distribusi Edu dari tiap cluster
raw_data.groupby('cluster')['Edu'].value_counts(normalize=True).unstack('Edu').plot.bar(stacked=True)

In [ ]:
# mengecek distribusi Years Employed dari tiap klaster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Years Employed')

In [ ]:
# mengecek distribusi Income dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Income')

In [ ]:
# mengecek distribusi Card Debt dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Card Debt')

In [ ]:
# mengecek distribusi Card Debt dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Other Debt')

In [ ]:
# mengecek distribusi Defaulted dari tiap cluster
raw_data.groupby('cluster')['Defaulted'].value_counts(normalize=True).unstack('Defaulted').plot.bar(stacked=True)

In [ ]:
# mengecek distribusi Defaulted dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'DebtIncomeRatio')

# Evaluasi Model

Untuk evaluasi model clustering kita dapat menggunakan average silhouette score

In [ ]:
visualizer = SilhouetteVisualizer(k_means, colors='yellowbrick')
visualizer.fit(data_for_model)
visualizer.show()

Mencari klaster optimal dengan Elbow Method

In [ ]:
# Evaluasi model menggunakan decision tree
visualizer = KElbowVisualizer(k_means, k=(2,12))
visualizer.fit(data_for_model)
visualizer.show()

In [ ]:
# menentukan jumlah kelompok data yang akan dibentuk
num_clusters = 6

# fit k-means model pada data
k_means_optimal = KMeans(n_clusters=num_clusters, random_state = 1000)
k_means_optimal.fit(data_for_model)

# label cluster
labels = k_means_optimal.labels_

# cek labels
print(labels)

Evaluasi kembali model menggunakan silhouette score

In [ ]:
visualizer = SilhouetteVisualizer(k_means_optimal, colors='yellowbrick')
visualizer.fit(data_for_model)
visualizer.show()

Bandingkan dengan model sebelumnya

In [ ]:
visualizer = SilhouetteVisualizer(k_means, colors='yellowbrick')
visualizer.fit(data_for_model)
visualizer.show()

Lakukan post-modeling analysis kembali berdasarkan model optimal

In [ ]:
# membuat kolom 'cluster' pada raw_data
raw_data['cluster'] = labels

# cek raw_data
raw_data.head()

In [ ]:
# mengecek distribusi age dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Age')

In [ ]:
# mengecek distribusi Edu dari tiap cluster
raw_data.groupby('cluster')['Edu'].value_counts(normalize=True).unstack('Edu').plot.bar(stacked=True)

In [ ]:
# mengecek distribusi Years Employed dari tiap klaster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Years Employed')

In [ ]:
# mengecek distribusi Income dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Income')

In [ ]:
# mengecek distribusi Card Debt dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Card Debt')

In [ ]:
# mengecek distribusi Card Debt dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'Other Debt')

In [ ]:
# mengecek distribusi Defaulted dari tiap cluster
raw_data.groupby('cluster')['Defaulted'].value_counts(normalize=True).unstack('Defaulted').plot.bar(stacked=True)

In [ ]:
# mengecek distribusi Defaulted dari tiap cluster
sns.boxplot(data = raw_data, x = 'cluster', y = 'DebtIncomeRatio')